# Built and compile the Unet

In [1]:
from utils.functions import r2_keras, msenonzero, data_split
from model.core import Unet
from tensorflow.keras.optimizers import Adam
from keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint
import numpy as np
import glob

Using TensorFlow backend.


In [2]:
unet = Unet()
opt = Adam(lr=1e-5) 

unet.compile(optimizer=opt, loss=msenonzero, metrics=[r2_keras, msenonzero])
unet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
model_input (InputLayer)        (None, 56, 120, 9)   0                                            
__________________________________________________________________________________________________
Block1_Conv1 (Conv2D)           (None, 56, 120, 128) 10496       model_input[0][0]                
__________________________________________________________________________________________________
Block1_Conv2 (Conv2D)           (None, 56, 120, 256) 295168      Block1_Conv1[0][0]               
__________________________________________________________________________________________________
Block1_MaxPool (MaxPooling2D)   (None, 28, 60, 256)  0           Block1_Conv2[0][0]               
____________________________________________________________________________________________

# Stage-1 training

### Load stage-1 data sets

In [13]:
x_files = sorted(glob.glob('sample_data/stage1/x/X_20*.npy'))
y_files = sorted(glob.glob('sample_data/stage1/y/Y_20*.npy'))


xtrain_files, ytrain_files = x_files[:14], y_files[:14]
xtrain = np.concatenate([ np.load(s) for s in xtrain_files], axis=0)
ytrain = np.concatenate([ np.load(s) for s in ytrain_files], axis=0)
# print(xtrain.shape, ytrain.shape)

# split into training, validation, and test sets
xtrain, ytrain, xvalid, yvalid = data_split(xtrain, ytrain, 0.9)
# print(xtrain.shape, ytrain.shape, xvalid.shape, yvalid.shape)

### Stage-1 training of the Unet

In [ ]:
csv_logger = CSVLogger( 'unet_stage1_log.csv', append=True, separator=';')
earlystopper = EarlyStopping(patience=15, verbose=1)
checkpointer = ModelCheckpoint('unet_checkpt_{val_loss:.2f}_{r2_keras:.2f}_stage1.h5', verbose=1, save_best_only=True)
unet.train(xtrain, ytrain, validation_data=(xvalid, yvalid), 
           batch_size=30, epochs=250, callbacks=[earlystopper, checkpointer, csv_logger], shuffle=True)

### Save stage-1 model weights

In [ ]:
unet.save_model('unet_stage1_model.h5')

### Generate predictions for evaluation

In [5]:
### Load testing data sets
xtest_files = x_files[14:]

### Predict using Unet
for x in xtest_files:
    xnow = np.load(x)
    pred = unet.predict(xtest)
    np.save('./stage1_output/pred_' + x)

(478, 35, 46, 9) (478, 35, 46, 1)


# Stage-2 training

### Load stage-2 data sets

In [15]:
x_files = sorted(glob.glob('sample_data/stage2/x/X_20*.npy'))
y_files = sorted(glob.glob('sample_data/stage2/y/Y_20*.npy'))
print(x_files, y_files)
xtrain_files, ytrain_files = x_files[:5], y_files[:5]
xtrain = np.concatenate([ np.load(s) for s in xtrain_files], axis=0)
ytrain = np.concatenate([ np.load(s) for s in ytrain_files], axis=0)
# print(xtrain.shape, ytrain.shape)

# split into training, validation, and test sets
xtrain, ytrain, xvalid, yvalid = data_split(xtrain, ytrain, 0.9)
# print(xtrain.shape, ytrain.shape, xvalid.shape, yvalid.shape)

['sample_data/stage2/x\\X_2014.npy', 'sample_data/stage2/x\\X_2015.npy', 'sample_data/stage2/x\\X_2016.npy', 'sample_data/stage2/x\\X_2017.npy', 'sample_data/stage2/x\\X_2018.npy', 'sample_data/stage2/x\\X_2019.npy', 'sample_data/stage2/x\\X_2020.npy'] ['sample_data/stage2/y\\Y_2014.npy', 'sample_data/stage2/y\\Y_2015.npy', 'sample_data/stage2/y\\Y_2016.npy', 'sample_data/stage2/y\\Y_2017.npy', 'sample_data/stage2/y\\Y_2018.npy', 'sample_data/stage2/y\\Y_2019.npy']


### Load the stage-1 model weights to the U-net model

In [ ]:
unet.load_weights('unet_stage1_model.h5')

### Stage-2 training of the Unet

In [ ]:
csv_logger = CSVLogger( 'unet_stage2_log.csv', append=True, separator=';')
earlystopper = EarlyStopping(patience=15, verbose=1)
checkpointer = ModelCheckpoint('unet_checkpt_{val_loss:.2f}_{r2_keras:.2f}_stage2.h5', verbose=1, save_best_only=True)
unet.train(xtrain, ytrain, validation_data=(xvalid, yvalid), 
           batch_size=30, epochs=250, callbacks=[earlystopper, checkpointer, csv_logger], shuffle=True)

### Save stage-2 model weights

In [ ]:
unet.save_model('unet_stage2_model.h5')

### Generate predictions for evaluation

In [16]:
### Load testing data sets
xtest_files = x_files[5:]
print(xtest_files)

### Predict using Unet
for x in xtest_files:
    xnow = np.load(x)
    pred = unet.predict(xtest)
    np.save('./stage2_output/pred_' + x)

['sample_data/stage2/x\\X_2019.npy', 'sample_data/stage2/x\\X_2020.npy']
